#Установка и мпорт библиотек

In [ ]:
!pip install nltk
!pip install langchain
!pip install spacy
!python -m spacy download ru_core_news_sm  # Загрузка модели для русского языка в spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 951.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 941.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Загрузка стоп-слов
nltk.download('punkt')     # Загрузка списка знаков пунктуации
nltk.download('stopwords') # Загрузка списка стоп-слов для различных языков

import spacy
# Загрузка русского лемматизатора из SpaCy
nlp = spacy.load("ru_core_news_sm")

import re  # Модуль для работы с регулярными выражениями

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity         # Импорт косинусного расстояния

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Импорт RecursiveCharacterTextSplitter из langchain для деления на chunks

import torch  # Импорт PyTorch

import gensim.downloader as api
# Загрузка модели GloVe (Global Vectors for Word Representation)
model = api.load("glove-wiki-gigaword-100")

import joblib

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Загружаем данные и словарь с аббревиатурами

In [ ]:
drk = pd.read_csv('merged_drk_df.csv')
cmk = pd.read_csv('merged_cmk_df.csv')
ftl = pd.read_csv('merged_ftl_df.csv')
otr = pd.read_csv('merged_otr_df.csv')
ft = pd.read_csv('merged_ft_df.csv')

In [ ]:
for df in [drk, cmk, ftl, otr, ft]:
  df.columns = ['filename', 'paragraphs', 'link', 'department']

In [ ]:
drk.head()

,filename,paragraphs,link,department
0,Вопросы по грузам с наложенным платежом,Возможно ли сделать корректировку плательщика ...,http://confluence/pages/viewpage.action?pageId...,DRK
1,Вопросы по грузам с наложенным платежом,"Клиенту необходим осмотр груза, а груз в ЗТУ. ...",http://confluence/pages/viewpage.action?pageId...,DRK
2,Вопросы по грузам с наложенным платежом,"Действия сотрудников, если клиент не согласен ...",http://confluence/pages/viewpage.action?pageId...,DRK
3,Вопросы по грузам с наложенным платежом,"Какие действия сотрудника, если в 1С указано т...",http://confluence/pages/viewpage.action?pageId...,DRK
4,Вопросы по грузам с наложенным платежом,"После оплаты НП и получения груза выяснилось, ...",http://confluence/pages/viewpage.action?pageId...,DRK


In [ ]:
abr = pd.read_csv('abbr.csv') #аббревиатуры
abr.columns = ['abbr', 'full'] # Переименование столбцов для удобства доступа
abr = abr.set_index('abbr')['full'].to_dict() # Преобразование в словарь для последующего использования при обучении модели

# Добавление дополнительной аббревиатуры в словарь
abr['FTL'] = 'Full Truck Load'
abr['Кыргызстан'] = 'Киргизия'
abr['МГГ'] = 'мелкогабаритный груз'
abr['НТМЦ'] = 'набор товарно-материальных ценностей'
abr['ЗУ'] = 'защитная упаковка'
abr['РТСД'] = 'радио терминал сбора данных'
abr['ШХ'] = 'штрих код'
abr['РС'] = 'регистр сведений'
abr['ТЧ'] = 'табличная часть'
abr['ПФ'] = 'печатная форма'
abr['ТСД'] = 'терминал сбора данных'
abr['НДС'] = 'налог'

#Пропишем необходимые функции для работы

## Предобработка данных

###Функция очистки текста

In [ ]:
def clean_text(text):
    # Удаление специальных символов с помощью регулярного выражения
    text = re.sub(r'[^\W\s]', ' ', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Лемматизация текста с использованием SpaCy
    doc = nlp(text)
    # Сбор лемматизированных токенов, исключая стоп-слова
    lemmatized_text = ' '.join(token.lemma_ for token in doc if not token.is_stop)
    return lemmatized_text

###Функция пополнения словаря модели синонимами для часто используемых слов

In [ ]:
synonyms_cache = abr  # Сразу добавляем аббревиатуры в кэш

def get_synonyms(word):
    if word not in synonyms_cache:  # Проверяем, есть ли слово в кэше синонимов
        try:
            sim_words = model.similar_by_word(word)  # Поиск схожих слов для данного слова в модели GloVe
            synonyms_cache[word] = [word for word, _ in sim_words][:1]  # Сохраняем только первый найденный синоним
        except KeyError:
            synonyms_cache[word] = []
    return synonyms_cache[word]  # Возвращаем список синонимов для данного слова из кэша

###Обогащение текста (контекста и запроса) синонимами

In [ ]:
def expand_with_synonyms(text):
    tokenized = nlp(text)  # Токенизация текста с помощью SpaCy
    expanded_text = []  #список для расширенного текста
    for token in tokenized:
        expanded_text.append(token.text)  # Добавление оригинального токена в расширенный текст
        if token.has_vector:
            synonyms = get_synonyms(token.text)  # Получение синонимов для текущего токена
            if synonyms:
                expanded_text.append(synonyms[0])  # Добавление первого найденного синонима
    return ' '.join(expanded_text)  # Возврат расширенного текста в виде строки

### Разбиваем тексты на чанки для лучшего распознавания

In [ ]:
def preprocess_documents(df):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=850, chunk_overlap=100)
    chunks = []  # Инициализация списка для хранения обработанных частей текста
    for entry in df:
        title = entry['filename']  # имя файла
        paragraphs = entry['paragraphs']  # текст
        split_chunks = text_splitter.split_text(paragraphs)  # Разделение абзацев на части
        for chunk in split_chunks:
            cleaned_chunk = clean_text(chunk)  # Очистка текста
            expanded_chunk = expand_with_synonyms(cleaned_chunk)  # Расширение текста с помощью синонимов
            chunks.append((title, expanded_chunk))  # Добавление кортежа (название файла, обработанный текст) в список
    return chunks

##Обучение векторизаторов для каждого отдела

### Функция векторизации TF-IDF

In [ ]:
def create_and_fit_vectorizer(chunks):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(chunk for _, chunk in chunks)  # Применение векторизатора к чанкам
    return vectorizer, X, chunks  # Возврат векторизатора, матрицы TF-IDF и списка чанков

### Функция векторизации для каждого отдела

In [ ]:
def model_wrapper(df, model_type):
    documents = df[['filename', 'paragraphs']].to_dict(orient='records')
    chunks = preprocess_documents(documents) # Предобработка
    vectorizer, X, chunks = create_and_fit_vectorizer(chunks) # Обучение TF-IDF векторизатора
    vectorizer_filename = f'vectorizer_{model_type}.joblib'
    X_filename = f'X_{model_type}.joblib' # Имена файлов для сохранения векторизатора и матрицы TF-IDF
    joblib.dump(vectorizer, vectorizer_filename)
    joblib.dump(X, X_filename) # сохранение
    return vectorizer, X, chunks    # Возврат векторизатора, матрицы TF-IDF и списка чанков

## Проверка работы модели

In [ ]:
#функция обработки запроса из набора тестовых запросов
def execute_queries(df, model_type, vectorizer, X, chunks):
    for item in knowledge_base["вопросы"]:
        if item['отдел'] == model_type:
            query = item['вопрос']  # Получаем текст запроса
            cleaned_query = clean_text(query)  # Очищаем и лемматизируем запрос
            expanded_query = expand_with_synonyms(cleaned_query)  # Расширяем запрос синонимами

            # Векторизуем расширенный запрос
            query_vec = vectorizer.transform([expanded_query])

            # Вычисляем косинусное расстояние между запросом и чанками
            results = cosine_similarity(X, query_vec).flatten()

            # Получаем индексы с наивысшей схожестью
            top_indices = results.argsort()[-5:][::-1]

            # Выводим информацию о запросе и результатах
            print("ДЕЛАЕМ ЗАПРОС: " + query)
            print("Материалы: " + str(item["материалы"]))
            print("Отдел: " + str(item["отдел"]))
            print()
            print("РЕЗУЛЬТАТЫ РАБОТЫ МОДЕЛИ: ")
            # Выводим топ-5 наиболее схожих записей
            for i in top_indices:
                similarity_score = results[i]
                print(f"Similarity: {similarity_score:.4f}")
                print(f"Title: {chunks[i][0]}")
                print(f"Text: {df[df['filename'] == chunks[i][0]]['paragraphs']}")
                print()
        else:
            continue

# Работа модели и оценка результатов

## Обучение векторизаторов

In [ ]:
# Определяем список названий моделей
model_types = ['DRK', 'CMK', 'FTL', 'OTR', 'FT']

# Определяем список датасетов
dfs = [drk, cmk, ftl, otr, ft]

In [ ]:
# Создаем словарь для хранения результатов обучения для каждой комбинации типа модели и объекта DataFrame
results = {}

# Итерируемся по каждому объекту DataFrame и типу модели
for df, model_type in zip(dfs, model_types):
    # Вызываем функцию-обертку для модели, чтобы получить векторизатор, матрицу X и чанки
    vectorizer, X, chunks = model_wrapper(df, model_type)

    # Сохраняем результаты в словаре
    results[(model_type, id(df))] = {
        'vectorizer': vectorizer,  # Векторизатор
        'X': X,  # Матрица признаков X
        'chunks': chunks  # Чанки
    }

## Создание новых df с чанками и векторами

In [66]:
# Итерируемся по каждому типу модели и соответствующему ей объекту DataFrame
for model_type, df in zip(model_types, dfs):
    # Получаем соответствующую матрицу X и chunks на основе типа модели и создаем DataFrame
    if model_type == 'DRK':
        X, chunks = results[('DRK', id(drk))]['X'], results[('DRK', id(drk))]['chunks']
        drk_chunks = pd.DataFrame(chunks, columns=['title', 'chunk'])
        drk_chunks['vectors'] = [list(vector) for vector in X]
    elif model_type == 'CMK':
        X, chunks = results[('CMK', id(cmk))]['X'], results[('CMK', id(cmk))]['chunks']
        cmk_chunks = pd.DataFrame(chunks, columns=['title', 'chunk'])
        cmk_chunks['vectors'] = [list(vector) for vector in X]
    elif model_type == 'FTL':
        X, chunks = results[('FTL', id(ftl))]['X'], results[('FTL', id(ftl))]['chunks']
        ftl_chunks = pd.DataFrame(chunks, columns=['title', 'chunk'])
        ftl_chunks['vectors'] = [list(vector) for vector in X]
    elif model_type == 'OTR':
        X, chunks = results[('OTR', id(otr))]['X'], results[('OTR', id(otr))]['chunks']
        otr_chunks = pd.DataFrame(chunks, columns=['title', 'chunk'])
        otr_chunks['vectors'] = [list(vector) for vector in X]
    elif model_type == 'FT':
        X, chunks = results[('FT', id(ft))]['X'], results[('FT', id(ft))]['chunks']
        ft_chunks = pd.DataFrame(chunks, columns=['title', 'chunk'])
        ft_chunks['vectors'] = [list(vector) for vector in X]
    else:
        continue


# соединяем DataFrame в один
chunked_df = pd.concat([drk_chunks, cmk_chunks, ftl_chunks, otr_chunks, ft_chunks], ignore_index=True)

In [67]:
chunked_df.head()

,title,chunk,vectors
0,Вопросы по грузам с наложенным платежом,...,"[ (0, 2)\t0.3035693623580902\n (0, 9)\t0.506..."
1,Вопросы по грузам с наложенным платежом,", and ...","[ (0, 2)\t0.7585070843093706\n (0, 3)\t0.554..."
2,Вопросы по грузам с наложенным платежом,", and ...","[ (0, 2)\t0.9598437348075876\n (0, 3)\t0.280..."
3,Вопросы по грузам с наложенным платежом,", and ...","[ (0, 2)\t0.7162724758802109\n (0, 3)\t0.697..."
4,Вопросы по грузам с наложенным платежом,", and ...","[ (0, 8)\t0.4431363590802261\n (0, 2)\t0.568..."


In [68]:
# сохраняем DataFrame в csv
chunked_df.to_csv('chunked_df.csv', index=False)

## Проверка работы векторизаторов

### Тестовые данные

In [ ]:
# список текстовых запросов с эталонными документами
knowledge_base = {
    "вопросы": [
        {
            "вопрос": "Как быть если заказчик требует водителя с мед книжкой?",
            "материалы": "На забор груза нужен водитель с мед. книжкой.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что входит в доставку в гипермаркеты?",
            "материалы": "Почему тариф на доставку в гипермаркет больше, чем стандартный.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как занести контакт в черный список?",
            "материалы": "Клиент просит отключить уведомления (E-mail; ВК; Viber; СМС; Автообзвон; Обзвон).html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Доступна ли перевозка грузов в Киргизию?",
            "материалы": "Курс лекций - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как найти груз по номеру интернет заказа?",
            "материалы": "Выставление счета в Информации по грузу.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как изменить страховую сумму при оформлении груза?",
            "материалы": "Оформление груза в СПО Пегас.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Возможна ли доставка день в день? Какие интервалы доставки?",
            "материалы": "Планирование доставки - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что делать, если клиент предоставил новый номер для оповещения?",
            "материалы": "Оповещение о прибытии груза (без доставки) - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Почему не выходят на печать чеки об оплате и ПКО не отображается в реестре?",
            "материалы": "Проверка и проведение оплаты в СПО - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как клиент может изменить дату авизации в личном кабинете?",
            "материалы": "Личный кабинет клиента (инструкция по применению) - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Клиент спрашивает, для чего мы запрашиваем письма от клиента, при предоставлении ему очередного спец условия",
            "материалы": "Заведение нового клиента и создание спец. условий - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что написать клиенту, который просит полный доступ к личному кабинету?",
            "материалы": "Шаблоны ответов при обработке почты lk@pecom.ru - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Какая упаковка используется при перевозке автомобильных стекол?",
            "материалы": "Перечень грузов, подлежащих обязательной дополнительной упаковке - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
            "вопрос": "Сколько времени может проработать подметальная машина?",
            "материалы": "Альбом _Оборудование и инструменты, используемые на участках склада_ - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
            "вопрос": "Что делать, если клиент отказывается получения сопроводительных документов?",
            "материалы": "Инструкция пользователя «Организация перевозки сопроводительных документов» - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
           "вопрос": "Какие есть размеры паллетных рам?",
           "материалы": "Альбом _Материалы, используемые на участках склада_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как осуществить Контроль выполнения НТМЦ?",
           "материалы": "Инструкция пользователя _Создание набора ТМЦ при помощи РТСД_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "При отправке грузов в Минск прямые или транзитные рейсы?",
           "материалы": "GLOBAL - Продуктовая книга - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Какие документы нужны для выдачи грузов в Беларусь (для физ. и юр. лиц)?",
           "материалы": "GLOBAL - Продуктовая книга - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Что делать, если клиент выбрал способ подписания Договорных документов на бумаге?",
           "материалы": "Инструкция пользователя «Размещение Договорных документов с Перевозчиком в интерфейсе ПО «Договоры с контрагентами» - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как провести обработку упрощенной выдачи по нескольким грузам?",
           "материалы": "Инструкция пользователя _Оформление выдачи груза в СПО_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Когда следует использовать нестандартную защитную упаковку?",
           "материалы": "Стандарт «Дополнительная упаковка груза» - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как следует грузить негабаритный груз?",
           "материалы": "Технологическая инструкция _Погрузка и разгрузка ТС с использованием КВП_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Что включает общий вид карточки контрагента?",
           "материалы": "Карточка контрагента - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как загрузить прайс лист по услугам FTL?",
           "материалы": "Загрузка прайса по услугам FTL в 1С_ TMS Логистика - в проде - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Что включает отчет по агентским вознаграждениям FTL?",
           "материалы": "Отчет по агентским вознаграждениям - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как поступить, если у заявки выставлен статус 'Подтверждение заявки с клиентом'?",
           "материалы": "Алгоритм действий сотрудников по работе с заявкой FTL в 1C_TMS (без документа Рейс) - в проде, версия от 21 года(возможно не актуально) - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как создать новую географическую зону?",
           "материалы": "Редактирование географических зон - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Какие бизнес правила страхования актуальны сегодня?",
           "материалы": "Услуги по страхованию груза и срока при перевозке груза по территории РФ и за ее пределами - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Кто изменял документ РасчитатьСпецУсловие_ЗаменаУслуг?",
           "материалы": "ОТР (RND-829).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Кто наполняет таблицу в базе данных api.RawLog?",
           "материалы": "ОТР (DARCH-1059).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Когда создана форма документа споВосстановлениеМДИ?",
           "материалы": "ОТР (DBUH-1198).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Как реализован механизм многопоточной обработки ЭТН?",
           "материалы": "ОТР (DED-51).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Кто на форму Обработка.ЗачетПереплат добавлял элементы ПериодВыставленияАктов и КонтрагентыНачинаютсяССимволов?",
           "материалы": "ОТР (DPEGAS-32409).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Что произойдет с просроченными грузами в ПВЗ?",
           "материалы": "ФТ (EPIC-11332).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Возможно ли создание большего количества характеров груза, чем в документе 'Настройка матрица соответствия внутреннего груза'?",
           "материалы": "ФТ (EPIC-10470).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Выставление документа Счет на оплату клиенту, при изменении данных о плательщике, городе оплаты и сумме?",
           "материалы": "ФТ (EPIC-10552).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Возможно ли изменить возвратные данные по грузу, до того, как груз станет возвратным?",
           "материалы": "ФТ (EPIC-10142).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Была ли доработка документа 'Инвентаризация на отправку'?",
           "материалы": "ФТ (EPIC-9495).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Была ли доработка документа 'Приемка'?",
           "материалы": "ФТ (EPIC-13571).html",
           "отдел": "FT"
        }
    ]
}


### Проверка результатов выдачи

In [ ]:
# Итерируемся по каждому типу модели и соответствующему ей объекту DataFrame
for model_type, df in zip(model_types, dfs):
    # Получаем соответствующий векторизатор, X и chunks на основе типа модели
    if model_type == 'DRK':
        vectorizer, X, chunks = results[('DRK', id(drk))]['vectorizer'], results[('DRK', id(drk))]['X'], results[('DRK', id(drk))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks) #обрабатываем каждый запрос в тестовых данных в зависимости от отдела и выводим результаты
    elif model_type == 'CMK':
        vectorizer, X, chunks = results[('CMK', id(cmk))]['vectorizer'], results[('CMK', id(cmk))]['X'], results[('CMK', id(cmk))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'FTL':
        vectorizer, X, chunks = results[('FTL', id(ftl))]['vectorizer'], results[('FTL', id(ftl))]['X'], results[('FTL', id(ftl))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'OTR':
        vectorizer, X, chunks = results[('OTR', id(otr))]['vectorizer'], results[('OTR', id(otr))]['X'], results[('OTR', id(otr))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'FT':
        vectorizer, X, chunks = results[('FT', id(ft))]['vectorizer'], results[('FT', id(ft))]['X'], results[('FT', id(ft))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    else:
        continue

ДЕЛАЕМ ЗАПРОС: Как быть если заказчик требует водителя с мед книжкой?
Материалы: На забор груза нужен водитель с мед. книжкой.html
Отдел: DRK

РЕЗУЛЬТАТЫ РАБОТЫ МОДЕЛИ: 
Similarity: 1.0000
Title: Вопросы по личному кабинету
Text: 39                                         Вопрос Ответ
40    Как подключить ЛК? 1. Зайти на сайт pecom.ru2....
41    Я не хочу отправлять вам скан своего паспорта,...
42    Я могу зарегистрироваться в ЛК, как физическое...
43    У меня есть ЛК для организации, мне нужно доба...
44    У меня несколько юр. лиц, я хочу сделать личны...
45    Во время работы в ЛК у меня появляется ошибка ...
46    Я не могу зайти в ЛК, появляется ошибка С элек...
47    Запрашиваю бух. документы и у меня появляется ...
48    Не работает ЛК С электронного адреса, к которо...
49    Не загружается информация в ЛК С электронного ...
50    Я хочу оформить отправку груза с НП, у меня ес...
51    Я хочу оформить отправку груза с НП, у меня ес...
52    Я могу оформить заявку с НП на сайте

# ГЕНЕРАЦИЯ

In [ ]:
#pip install gpt-2-simple

In [ ]:
#import gpt_2_simple as gpt2
#model_name = "124M"
#gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 2.29Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 3.83Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 1.22Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.5Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 406Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.71Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 7.26Mit/s]


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
"""text_data = {}
for index, row in df.iterrows():
    filename = row['filename']
    paragraphs = row['paragraphs']
    text_data[filename] = "\n".join(paragraphs)"""

In [ ]:
"""import os

# Choose an empty directory or create a new one
file_path = "drive/MyDrive/text_files"

# Ensure the output directory exists within the mounted drive; create it if it doesn't
os.makedirs(output_dir, exist_ok=True)

# Write concatenated text to individual text files in the specified directory
for filename, text in text_data.items():
    file_path = os.path.join(output_dir, filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)"""

In [ ]:
"""# Define the path to your dataset (directory containing your text files)
data_dir = "content"

# Choose the GPT-2 model size (124M, 355M, 774M, 1558M)


# Fine-tune GPT-2 on your dataset
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=data_dir,
              model_name=model_name,
              steps=1000,  # Number of training steps
              restore_from='fresh',  # Or 'latest' to resume training from existing checkpoint
              run_name='fine_tuned_model'  # Name of the run, used to save checkpoints
              )

# Save the fine-tuned model checkpoint
gpt2.save_gpt2(sess, run_name='fine_tuned_model')"""